In [2]:
import cv2
import numpy as np
import time
import csv

# Define color ranges in HSV for different ball colors
color_ranges = {
    "green": ([36, 100, 100], [89, 255, 255]),
    "orange": ([10, 100, 20], [25, 255, 255]),
    "yellow": ([20, 100, 100], [35, 255, 255]),
}

def identify_ball_color(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    for color, (lower, upper) in color_ranges.items():
        mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((15, 15), np.uint8))  # Larger kernel to reduce noise
        if cv2.countNonZero(mask) > 500:  # Adjust the threshold based on the size of the ball in the frame
            return color
    return "unknown"

def process_frame(frame, prev_quadrants, log, fps, frame_count):
    # Convert the frame to the HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Define the red color range in HSV
    lower_red1 = np.array([0, 70, 50])
    upper_red1 = np.array([10, 255, 255])
    lower_red2 = np.array([170, 70, 50])
    upper_red2 = np.array([180, 255, 255])

    # Create masks for red color
    mask1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask = mask1 | mask2

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    current_quadrants = [None, None, None, None]  # For storing current quadrant states

    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
        if len(approx) == 4:  # Checking for a quadrilateral
            cv2.drawContours(frame, [approx], 0, (0, 255, 0), 3)
            
            # Get the coordinates of the quadrilateral
            pts = approx.reshape(4, 2)
            pts = sorted(pts, key=lambda x: (x[1], x[0]))  # Sort by y, then by x
            top_left, top_right = sorted(pts[:2], key=lambda x: x[0])
            bottom_left, bottom_right = sorted(pts[2:], key=lambda x: x[0])
            
            # Calculate midpoints
            mid_top = (top_left + top_right) // 2
            mid_bottom = (bottom_left + bottom_right) // 2
            mid_left = (top_left + bottom_left) // 2
            mid_right = (top_right + bottom_right) // 2
            center = (mid_left + mid_right) // 2
            
            # Calculate bounding box coordinates for quadrants
            width = (top_right[0] - top_left[0]) // 2
            height = (bottom_left[1] - top_left[1]) // 2
            
            quadrants = [
                (top_left, (top_left[0] + width, top_left[1] + height)),
                ((top_left[0] + width, top_left[1]), (center[0] + width, center[1])),
                ((top_left[0], top_left[1] + height), (center[0], center[1] + height)),
                (center, bottom_right)
            ]
            
            # Draw lines to divide into quadrants
            cv2.line(frame, tuple(mid_top), tuple(mid_bottom), (255, 0, 0), 2)
            cv2.line(frame, tuple(mid_left), tuple(mid_right), (255, 0, 0), 2)

            for i, (pt1, pt2) in enumerate(quadrants):
                x1, y1 = pt1
                x2, y2 = pt2
                if x1 < x2 and y1 < y2:  # Ensure the coordinates are valid
                    quadrant = frame[y1:y2, x1:x2]
                    gray_quadrant = cv2.cvtColor(quadrant, cv2.COLOR_BGR2GRAY)
                    non_zero_count = cv2.countNonZero(gray_quadrant)
                    current_quadrants[i] = non_zero_count

                    if prev_quadrants[i] is not None:
                        # Compare current quadrant with previous quadrant
                        if non_zero_count > prev_quadrants[i] + 500:  # Threshold to detect significant change
                            ball_color = identify_ball_color(quadrant)
                            if ball_color != "unknown":
                                timestamp = time.strftime('%H:%M:%S', time.gmtime(frame_count / fps))
                                log.append((f"Quadrant {i+1}", ball_color, "placed", timestamp))
                        elif non_zero_count < prev_quadrants[i] - 500:  # Threshold to detect significant change
                            ball_color = identify_ball_color(quadrant)
                            if ball_color != "unknown":
                                timestamp = time.strftime('%H:%M:%S', time.gmtime(frame_count / fps))
                                log.append((f"Quadrant {i+1}", ball_color, "removed", timestamp))

    return frame, current_quadrants

def main(video_path, output_csv):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = 0

    prev_quadrants = [None, None, None, None]
    log = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Process the frame
        processed_frame, current_quadrants = process_frame(frame, prev_quadrants, log, fps, frame_count)

        # Update previous quadrants
        prev_quadrants = current_quadrants

        # Display the frame
        cv2.imshow('Processed Frame', processed_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Write log to CSV
    with open(output_csv, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Quadrant', 'Ball Color', 'Action', 'Timestamp'])
        for entry in log:
            writer.writerow(entry)

    print(f"Output saved to {output_csv}")

if __name__ == "__main__":
    video_path = 'input_video.mp4'
    output_csv = 'output_log.csv'
    main(video_path, output_csv)


Output saved to output_log.csv
